In [1]:
import pandas

In [2]:
import random

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
abalones = pandas.read_excel("dataset_processado.xlsx");
test = pandas.read_excel("app_processado.xlsx");

accuracy = {}; #Grupo de parâmetros: acurácia

In [5]:
#Parâmetros de teste
numReps = 10;
numChanges = 1;
randomSplit = 0.2;


In [ ]:
#Geração Aleatória
for n in range(numChanges):
    weights = [random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(), random.randrange(3, 50, 2)]    
    accuracyResults = [];

    for n in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    meanAcc = sum(accuracyResults)/len(accuracyResults)
    print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7])
         )
    accuracy[str(weights)] = meanAcc

In [ ]:
#Forward selection

def forwardSelection(baseWeights, baseAcc):
    m = 0;
    accs = [];
    for col in abalones.columns[:8]:
        weights = baseWeights.copy();
        weights[m] = 1
        accuracyResults = [];
    
        for num in range(numReps):
            knn = KNeighborsClassifier(n_neighbors = weights[8]);

            #Selecionando índices para teste e treino
            trainingIndexes = [];
            testIndexes = [];
            n = 0;
            for item in abalones["sex"]:
                if random.random() > randomSplit:
                    trainingIndexes.append(n)
                else:
                    testIndexes.append(n)
                n+=1

            #Criando os dataframes de treino e teste e separando os classificadores
            testData = abalones.drop(trainingIndexes);
            testClasses = testData["type"];
            testData = testData.drop(columns=["type"]);

            trainingData = abalones.drop(testIndexes);
            classifications = trainingData["type"]
            trainingData = trainingData.drop(columns=["type"])

            #Aplicando pesos
            n = 0;
            for col in trainingData.columns:
                trainingData[col] = trainingData[col]*weights[n]
                testData[col] = testData[col]*weights[n]
                n+=1

            #Aplicando K-NN
            knn.fit(trainingData, classifications)

            results = knn.predict(testData);

            #Testando acurácia
            n = 0;
            correctPredictions = 0;
            for item in testClasses:
                if item == results[n]:
                    correctPredictions += 1;
                n+=1  

            acc = correctPredictions/len(results)
            accuracyResults.append(acc)

        meanAcc = sum(accuracyResults)/len(accuracyResults)
        '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
        print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
              + "\nSexo: " + str(weights[0]) 
              + "\nComprimento: " + str(weights[1])
              + "\nDiâmetro: " + str(weights[2])
              + "\nAltura: " + str(weights[3])
              + "\nPeso cheio: " + str(weights[4])
              + "\nPeso vazio: " + str(weights[5])
              + "\nPeso das vísceras: " + str(weights[6])
              + "\nPeso da concha: " + str(weights[7]) + "\n\n"
             )'''
        accs.append(meanAcc)
        m+=1
    
    #print("Acurácias finais: " + str(accs) + '\n\n')
    
    #Índice do melhor resultado
    ind = accs.index(max(accs))
    
    #Conferindo se houve melhora nessa recursão, caso não, já retorna a última lista de parâmetros
    if accs[ind] <= baseAcc:
        print("Resultado final: " + str(accs[ind]) + "%")
        return baseWeights
    #Caso haja melhora, a função será chamada novamente, com uma nova base de pesos e com a melhor acurácia
    else:
        newWeights = baseWeights.copy();
        newWeights[ind] = 1;
        return forwardSelection(newWeights, accs[ind])
    
    
for neighs in [5,15,25,50]:
    print(forwardSelection([0,0,0,0,0,0,0,0,neighs], 0))

#bestSet = forwardSelection([0,0,0,0,0,0,0,0,5], 0)

In [6]:
numPop = 50;
population = {}

for n in range(numPop):
    population[n] = [0,0,0,0,0,0,0,0,3]

In [7]:
def generatePop():
    for m in range(numPop):
        newPop = [];
        for n in range(8):
            newPop.append(random.randrange(101))
        newPop.append(random.randrange(3,100))
        population[m] = newPop;

In [8]:
def checkFitness(weights):
    accuracyResults = [];
    for num in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    return sum(accuracyResults)/len(accuracyResults)
    '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7]) + "\n\n"
         )'''

In [9]:
def getFitness():
    results = {};
    for n in range(numPop):
        results[n] = checkFitness(population[n]);
        print ("Checando a adequação de "+ str(n+1) + " de " + str(numPop) )
    return results

In [10]:
def selection(fitness):
    #Normalizando a chance de reprodução de acordo com o fitness
    best = max(fitness.values())
    worst =  min(fitness.values())
    for k in fitness.keys():
        fitness[k] = (fitness[k]-worst) / (best-worst)
    
    #Agora, com a tabela normalizada, temos a probabilidade de reprodução entre 0 e 1. Enquanto o tamanho da população não for
    #50% do original, continuamos removendo

    removed = 0;
    tries = 0;
    #print(numPop/2)
    while removed < (numPop/2) and tries < 2000:
        ind = random.randrange(numPop);
        #print("Índice: " + str(ind) + "\nParâmetros: " + str(population[ind]));
        #Ignorar se já foi removido
        if population[ind] == 0:
            tries+=1
            continue;
        else:
            if random.random() > fitness[ind]:
                #print("População removida.")
                removed+=1
                #print("Já removidos: " + str(removed))
                population[ind] = 0;
            else:
                tries+=1
    


In [11]:
def crossover():
    newborns = 0;
    while newborns < numPop/2:
        #Selecionar aleatoriamente dois indivíduos
        ind1 = random.randrange(numPop);
        ind2 = random.randrange(numPop);
        
        while population[ind1] == 0:
            ind1 = random.randrange(numPop);
        while population[ind2] == 0 or ind2 == ind1:
            ind2 = random.randrange(numPop);
            
        newborns+=1
        

In [12]:
generatePop()

In [13]:
fitness = getFitness()

Checando a adequação de 1 de 50
Checando a adequação de 2 de 50
Checando a adequação de 3 de 50
Checando a adequação de 4 de 50
Checando a adequação de 5 de 50
Checando a adequação de 6 de 50
Checando a adequação de 7 de 50
Checando a adequação de 8 de 50
Checando a adequação de 9 de 50
Checando a adequação de 10 de 50
Checando a adequação de 11 de 50
Checando a adequação de 12 de 50
Checando a adequação de 13 de 50
Checando a adequação de 14 de 50
Checando a adequação de 15 de 50
Checando a adequação de 16 de 50
Checando a adequação de 17 de 50
Checando a adequação de 18 de 50
Checando a adequação de 19 de 50
Checando a adequação de 20 de 50
Checando a adequação de 21 de 50
Checando a adequação de 22 de 50
Checando a adequação de 23 de 50
Checando a adequação de 24 de 50
Checando a adequação de 25 de 50
Checando a adequação de 26 de 50
Checando a adequação de 27 de 50
Checando a adequação de 28 de 50
Checando a adequação de 29 de 50
Checando a adequação de 30 de 50
Checando a adequaçã

In [14]:
selection(fitness)

In [15]:
crossover()

17
0
3
26
28
31
3
34
16
38
37
16
11
17
41
17
37
45
38
0
4
29
11
24
29
33
24
35
3
38
28
24
28
36
33
11
22
16
37
9
41
29
45
9
29
34
36
3
32
0
